In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 17 18:43:07 2021

@author: sunym
"""
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pickle
import pandas as pd
X = pd.read_csv("heart.csv")


def validation(X,Y, TestSize = 0.2,epo = 30):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    #build the regular nn
    Input = tf.keras.layers.Input(shape = (15,), dtype = tf.float32)

    X = tf.keras.layers.Dense(160,activation = 'relu')(Input)
    X = tf.keras.layers.Dropout(rate = 0.1)(X)
    X = tf.keras.layers.Dense(120,activation = 'relu')(X)
    X = tf.keras.layers.Dropout(rate = 0.0)(X)
    X = tf.keras.layers.Dense(160,activation = 'relu')(X)
    X = tf.keras.layers.Dropout(rate = 0.3)(X)
    y_pred = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)

    REmodel = tf.keras.models.Model(inputs=[Input], outputs=y_pred)
    REmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0007),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    history = REmodel.fit(X_train, Y_train,epochs=epo,
                          validation_data=(X_test,Y_test),batch_size = 64,verbose = 1)

    _, acc_re = REmodel.evaluate(X_test, Y_test,verbose = 0)
    pre_re = REmodel.predict(X_test)
    predict1prob_re = pre_re[:,0]
    pre_re = (pre_re >=0.5).astype(int)
    tnre, fpre, fnre, tpre = confusion_matrix(Y_test,pre_re).ravel()
    sensitivity_re = tpre/(tpre+fnre)
    specificity_re = tnre/(tnre+fpre)
    avg_accre = (sensitivity_re + specificity_re)/2

    fpr_re, tpr_re, thresholds_re = metrics.roc_curve(Y_test,predict1prob_re)
    AUC_re = metrics.auc(fpr_re, tpr_re)

    del REmodel
    tf.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()

    return {'DNN Accuracy':acc_re, 'DNN C Stat':AUC_re,
            'DNN Sensitivity':sensitivity_re,
            'DNN Specificity':specificity_re,'DNN Balance Accuracy':avg_accre}


#X = np.genfromtxt("correctModelSR32_x.csv",delimiter=",",skip_header=1)
#Y = np.genfromtxt("correctModelSR32_y.csv",delimiter=",",skip_header=1)

X['Sex'] = pd.get_dummies(X['Sex'])['F']
X['ATA'] = pd.get_dummies(X['ChestPainType'])['ATA']
X['NAP'] = pd.get_dummies(X['ChestPainType'])['NAP']
X['ChestPainType'] = pd.get_dummies(X['ChestPainType'])['ASY']
X['LVH'] = pd.get_dummies(X['RestingECG'])['LVH']
X['RestingECG'] = pd.get_dummies(X['RestingECG'])['Normal']
X['ExerciseAngina'] = pd.get_dummies(X['ExerciseAngina'])['Y']
X['Flat'] = pd.get_dummies(X['ST_Slope'])['Flat']
X['ST_Slope'] = pd.get_dummies(X['ST_Slope'])['Down']

Y = X['HeartDisease']
X = X.drop(['HeartDisease'],axis = 1)

X = np.array(X)

Times = 25
DNNAcc = np.zeros((Times,))
DNNC = np.zeros((Times,))
DNNSens = np.zeros((Times,))
DNNSpec= np.zeros((Times,))
DNNBalanceAcc = np.zeros((Times,))
for i in range(0,Times):
    Results = validation(X,Y,epo = 60)
    DNNAcc[i] = Results['DNN Accuracy']
    DNNC[i] = Results['DNN C Stat']
    DNNSens[i] = Results['DNN Sensitivity']
    DNNSpec[i]= Results['DNN Specificity']
    DNNBalanceAcc[i] = Results['DNN Balance Accuracy']
    print(i)

Results_all = {'DNN Acc':DNNAcc,'DNN C Stat': DNNC,
                 'DNN Sensitivity':DNNSens,'DNN Specificity':DNNSpec,
                 'DNN Balance Accuracy':DNNBalanceAcc}
pickle.dump(Results_all,open("DNN_sr32.p","wb"))
Average_Results = {k: np.mean(v) for k, v in Results_all.items()}
Average_Results

Epoch 1/60
12/12 [==============================] - 2s 24ms/step - loss: 3.9289 - accuracy: 0.5586 - val_loss: 0.6374 - val_accuracy: 0.7609
Epoch 2/60
12/12 [==============================] - 0s 7ms/step - loss: 2.4453 - accuracy: 0.5913 - val_loss: 0.9639 - val_accuracy: 0.6087
Epoch 3/60
12/12 [==============================] - 0s 8ms/step - loss: 1.8602 - accuracy: 0.6049 - val_loss: 0.7332 - val_accuracy: 0.6576
Epoch 4/60
12/12 [==============================] - 0s 6ms/step - loss: 1.3796 - accuracy: 0.5940 - val_loss: 0.6538 - val_accuracy: 0.6957
Epoch 5/60
12/12 [==============================] - 0s 7ms/step - loss: 1.0040 - accuracy: 0.6322 - val_loss: 0.5702 - val_accuracy: 0.7120
Epoch 6/60
12/12 [==============================] - 0s 7ms/step - loss: 0.9422 - accuracy: 0.6226 - val_loss: 0.5370 - val_accuracy: 0.7228
Epoch 7/60
12/12 [==============================] - 0s 7ms/step - loss: 0.8464 - accuracy: 0.6240 - val_loss: 0.5477 - val_accuracy: 0.7120
Epoch 8/60
12/12 [=

{'DNN Acc': 0.7719565272331238,
 'DNN C Stat': 0.8393256876582688,
 'DNN Sensitivity': 0.8320234327399172,
 'DNN Specificity': 0.6997999218933042,
 'DNN Balance Accuracy': 0.7659116773166107}